In [1]:
#from pyDeepInsight import ImageTransformer, LogScaler
from image_transformer import ImageTransformer, LogScaler
import numpy as np
from tensorflow.keras import datasets, layers, models

from sklearn import tree
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
import time
from sklearn.decomposition import PCA

from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

from sklearn.model_selection import train_test_split


In [2]:
# Stop GPU cuda, to avoid cuda error imcomatibale between keras library and cuda nn
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [3]:
# load dataset
x_train=np.load("X_train.npy")
    
scaler=StandardScaler()
scaler.fit(x_train)
x_train=scaler.transform(x_train)
#===
y_train=np.load("y_train.npy")

# -------------------------------------

x_test=np.load("X_test.npy")

#====z_score normalization
scaler2=StandardScaler()
scaler2.fit(x_test)
x_test=scaler2.transform(x_test)
#====
y_test=np.load("y_test.npy")

print(x_train.shape, x_test.shape)

(8346, 12) (3577, 12)


In [4]:
all_data_train=np.concatenate((x_train, x_test), axis= 0)
all_data_y=np.concatenate((y_train, y_test), axis= 0)
all_data_train.shape

(11923, 12)

In [5]:
t1=time.time()

# normalize data
ln = LogScaler()
all_data_train=ln.fit_transform(all_data_train)
#x_test=ln.fit_transform(x_test)
#all_data_train1=all_data_train[:50000,]
#all_data_train2=all_data_train[:50000,]

# transform data to image form towork in CNN
it = ImageTransformer(feature_extractor='tsne', 
                      pixels=30, random_state=1701, 
                      n_jobs=-1)

#all_data_train1 = it.fit_transform(all_data_train1)
all_data_train = it.fit_transform(all_data_train)
#x_test = it.fit_transform(x_test)
#print(all_data_train1.shape, all_data_train2.shape)

In [6]:
#all_data_train=np.concatenate((all_data_train1, all_data_train2), axis=0)
all_data_train.shape

(11923, 30, 30, 3)

In [7]:

X_train, X_test, y_train, y_test = train_test_split(
    all_data_train, all_data_y, test_size=0.20, random_state=42)
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(9538, 30, 30, 3) (2385, 30, 30, 3)
(9538,) (2385,)


In [8]:
# Create the convolutional base
input_shape=X_train[0].shape

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))

In [9]:
# Add Dense layers on top
model.add(layers.Flatten())
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 12, 12, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 32)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 4, 32)          9248      
_________________________________________________________________
flatten (Flatten)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 32)                1

In [10]:

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit the keras model on the dataset
model.fit(X_train, y_train, epochs=20, batch_size=256)
# evaluate the keras model
_, accuracy = model.evaluate(X_train, y_train)
print('Accuracy training : %.2f' % (accuracy*100))

t2=time.time()

print ("time consumption : ", t2-t1)

Epoch 1/20
38/38 [==============================] - 5s 129ms/step - loss: 0.3042 - accuracy: 0.8855
Epoch 2/20
38/38 [==============================] - 4s 112ms/step - loss: 0.0634 - accuracy: 0.9609
Epoch 3/20
38/38 [==============================] - 4s 109ms/step - loss: 0.0089 - accuracy: 1.0000
Epoch 4/20
38/38 [==============================] - 4s 112ms/step - loss: 1.0426e-04 - accuracy: 1.0000
Epoch 5/20
38/38 [==============================] - 4s 108ms/step - loss: 4.2709e-05 - accuracy: 1.0000
Epoch 6/20
38/38 [==============================] - 5s 131ms/step - loss: 2.9237e-05 - accuracy: 1.0000
Epoch 7/20
38/38 [==============================] - 5s 129ms/step - loss: 2.1633e-05 - accuracy: 1.0000
Epoch 8/20
38/38 [==============================] - 5s 129ms/step - loss: 1.6751e-05 - accuracy: 1.0000
Epoch 9/20
38/38 [==============================] - 5s 130ms/step - loss: 1.3341e-05 - accuracy: 1.0000
Epoch 10/20
38/38 [==============================] - 5s 127ms/step - loss: 1

In [11]:

#yhat_classes = model.predict_classes(X_test, verbose=0)
yhat_classes = (model.predict(X_test) > 0.5).astype("int32")
#conf_matrix = confusion_matrix(y_true=y_test, y_pred=dt_pred)
print('Precision: %.3f' % precision_score(y_test, yhat_classes))
print('Recall: %.3f' % recall_score(y_test, yhat_classes))
print('F1 Score: %.3f' % f1_score(y_test, yhat_classes))
print('Accuracy: %.3f' % accuracy_score(y_test, yhat_classes))


Precision: 1.000
Recall: 1.000
F1 Score: 1.000
Accuracy: 1.000
